In [ ]:
import os
import xml.etree.ElementTree as ET

import numpy as np
import rasterio
import rasterio.merge
import fiona
from shapely.geometry import shape

from shareloc.geomodels import GeoModel
from shareloc.geofunctions.localization import Localization
from shareloc.dtm_reader import dtm_reader
from shareloc.geofunctions.dtm_intersection import DTMIntersection

In [ ]:
# Mode is either "SRTM" or "COPERNICUS"
mode = "COPERNICUS"
path_dim = "/work/CAMPUS/etudes/3D/Development/malinoro/Pleiades/Yosemite/2017-05-02_18h49m264/IMG_PHR1B_P_001/DIM_PHR1B_P_201705021849264_SEN_PWH-8376_2024-05-03_08-08-38_1039-001.XML"
output_path = "/work/CAMPUS/etudes/3D/Development/malinoro/CARS_output/Yosemite/COPERNICUS/COPERNICUS_merged.tif"

dem_root = {"SRTM": "/work/datalake/static_aux/MNT/SRTM_30_hgt/", "COPERNICUS": "/work/datalake/static_aux/MNT/COP-DEM_GLO-30-DGED_extracted/"}
dem_folder = dem_root[mode]

In [ ]:
tree = ET.parse(path_dim)
root = tree.getroot()

In [ ]:
lat_min, lat_max, lon_min, lon_max = np.inf, -np.inf, np.inf, -np.inf

for vertex in root.find("Dataset_Content").find("Dataset_Extent").findall("Vertex"):
    lat_min = min(lat_min, float(vertex.find("LAT").text))
    lat_max = max(lat_max, float(vertex.find("LAT").text))
    lon_min = min(lon_min, float(vertex.find("LON").text))
    lon_max = max(lon_max, float(vertex.find("LON").text))

list_lat = []
list_lon = []
list_dem = []
for lat in range(np.floor(lat_min).astype(int), np.ceil(lat_max).astype(int)):
    if lat >= 0:
        dem_lat = f"N{lat:02d}"
    else:
        dem_lat = f"S{-lat:02d}"
    list_lat += [dem_lat]
    
    for lon in range(np.floor(lon_min).astype(int), np.ceil(lon_max).astype(int)):
        if lon >= 0:
            dem_lon = f"E{lon:03d}"
        else:
            dem_lon = f"W{-lon:03d}"
        
        list_lon += [dem_lon]
        if mode == "SRTM":
            list_dem += [f"{dem_lat}{dem_lon}.hgt"]
        elif mode == "COPERNICUS":
            list_dem += [f"{dem_lon}/Copernicus_DSM_10_{dem_lat}_00_{dem_lon}_00/DEM/Copernicus_DSM_10_{dem_lat}_00_{dem_lon}_00_DEM.tif"]
        else:
            raise RuntimeError("Mode '{mode}' should be either 'SRTM' or 'COPERNICUS'")
print(f"LAT {lat_min} <--> {lat_max} ==  {list_lat}")
print(f"LON {lon_min} <--> {lon_max} ==  {list_lon}")
print(list_dem)

In [ ]:
for k in list_dem:
    if mode == "SRTM":
        msg_error = "\nTry with Copernicus DEM?"
    else:
        msg_error = ""
    assert os.path.exists(os.path.join(dem_folder, k)), f"{os.path.join(dem_folder, k)} does not exists{msg_error}"

In [ ]:
if not os.path.exists(os.path.dirname(output_path)):
    raise RuntimeError(f"Output directory does not exists: {os.path.dirname(output_path)}.\nConsider creating it with:\n\nos.makedirs('{os.path.dirname(output_path)}')\n")

In [ ]:
list_hgt = []
for dem_file in list_dem:
    list_hgt += [rasterio.open(os.path.join(dem_folder, dem_file))]

In [ ]:
no_data_list = np.unique([k.meta['nodata'] for k in list_hgt if k.meta['nodata'] is not None])
if no_data_list.size == 1:
    no_data = no_data_list[0]
else:
    print(f"Could not identify no_data: {no_data_list}. Please set var 'no_data' manually")

In [ ]:
no_data = None

In [ ]:
data, transform = rasterio.merge.merge(list_hgt, nodata=no_data)
data = np.squeeze(data)

In [ ]:
data.shape, transform

In [ ]:
profile = list_hgt[0].profile
profile.update(height=data.shape[0], width=data.shape[1], transform=transform, driver="GTiff")

with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(data, 1)

# Getting rows and columns based on shapefile

### Inputs
Provided a RPC file and a vector path (polygon .shp), computes the rows and cols that will be extracted with OTB. We add a margin on the sides. 

In [ ]:
rpc_path = "/work/CAMPUS/etudes/3D/Development/malinoro/Pleiades/Yosemite/2017-05-02_18h49m264/IMG_PHR1B_P_001/RPC_PHR1B_P_201705021849264_SEN_PWH-8376_2024-05-03_08-08-38_1039-001.XML"
vector_path = "/work/CAMPUS/users/malinoro/ETUDES/Glacier/DEM/Graasubreen.shp"

margin = 3000 # pixels

In [ ]:
rpc = GeoModel(rpc_path,  geomodel_type="RPC")
dtm_read = dtm_reader(output_path)
dtm = DTMIntersection(dtm_read.epsg, dtm_read.alt_data, dtm_read.nb_rows, dtm_read.nb_columns, dtm_read.transform)
loc = Localization(rpc, dtm)

In [ ]:
polys = []
vec_file = fiona.open(vector_path, "r")
for feat in vec_file:
    polys.append(shape(feat["geometry"]))
minx, miny, maxx, maxy = polys[0].bounds

row_max, col_max = np.floor(rpc.rowmax).astype(int), np.floor(rpc.colmax).astype(int)
ulx, uly, lrx, lry = col_max, row_max, 0, 0
for lon in [minx, maxx]:
    for lat in  [miny, maxy]:
        row, col = loc.inverse(lon, lat)[0:2]
        ulx = min(ulx, np.floor(col)[0].astype(int))
        uly = min(uly, np.floor(row)[0].astype(int))
        
        lrx = max(lrx, np.ceil(col)[0].astype(int))
        lry = max(lry, np.ceil(row)[0].astype(int))
        
tmp = f"""({ulx}, {uly}) --------> x
 | 
 | 
 | 
 |
 ▼                ({lrx}, {lry})
"""
print(tmp)

In [ ]:
otb_ulx = max(ulx-margin, 0)
otb_uly = max(uly-margin, 0)

otb_lrx = min(lrx+margin, col_max)
otb_lry = min(lry+margin, row_max)

print(f"otbcli_ExtractROI -mode extent -mode.extent.ulx {otb_ulx} -mode.extent.uly {otb_uly} -mode.extent.lrx {otb_lrx} -mode.extent.lry {otb_lry} -in -out ")